# StackExchange questions and Tags - Modélisation

This Notebook is about Stackoverflow questions tag prediction 
- I start by importing libraries and Data 
- I make a first unsupervised approch with LDA
- I make different approaches for feature engineering : Bag of Words and TF-IDF
- I finally try supervised approch to classify and predict the tags

## Libraries and frameworks

In [1]:
!pip install tqdm
!pip install flake8 --ignore-installed
!pip install pycodestyle_magic
!pip install wordcloud

  Using cached https://files.pythonhosted.org/packages/f8/1f/7ea40d1e4146ea55dbab41cda1376db092a75794914169aabd7e8d7a7def/flake8-3.7.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0e/0c/04a353e104d2f324f8ee5f4b32012618c1c86dd79e52a433b64fceed511b/pycodestyle-2.5.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/87/89/479dc97e18549e21354893e4ee4ef36db1d237534982482c3681ee6e7b57/mccabe-0.6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/84/f2/ed0ffb887f8138a8fe5a621b8c0bb9598bfb3989e029f6c6a85ee66628ee/pyflakes-2.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/c6/44694103f8c221443ee6b0041f69e2740d89a25641e62fb4f2ee568f2f9c/entrypoints-0.3-py2.py3-none-any.whl


ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.


In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
import matplotlib.cm as cm
import numpy as np

from tqdm import tqdm

from bs4 import BeautifulSoup
import nltk
from sklearn.decomposition import LatentDirichletAllocation
import heapq

In [3]:
%load_ext pycodestyle_magic
%pycodestyle_on
%matplotlib inline

plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

## Data

In [4]:
df = pd.read_csv('C:/Users/ayoub/OCP6 Data/df.csv')

In [5]:
df

,Body,Title,Tags,AnswerCount,FavoriteCount,tokens,title_tokens,tags,ntags
0,I want to use a track-bar to change a form's o...,Convert Decimal to Double,<c#><floating-point><type-conversion><double><...,13,50,"['want', 'use', 'track-bar', 'chang', ""form'"",...","['convert', 'decim', 'doubl']","['c#', 'floating-point', 'type-conversion', 'd...",5
1,I have an absolutely positioned div containing...,Percentage width child element in absolutely p...,<html><css><internet-explorer-7>,6,11,"['absolut', 'posit', 'div', 'contain', 'sever'...","['percentag', 'width', 'child', 'element', 'ab...","['html', 'css', 'internet-explorer-7']",3
2,Given a DateTime representing a person's birth...,How do I calculate someone's age in C#?,<c#><.net><datetime>,61,443,"['given', 'datetim', 'repres', ""person'"", 'bir...","['calcul', ""someone'"", 'age', 'c#?']","['c#', '.net', 'datetime']",3
3,"Given a specific DateTime value, how do I disp...",Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-s...,37,540,"['given', 'specif', 'datetim', 'value,', 'disp...","['calcul', 'rel', 'time', 'c#']","['c#', 'datetime', 'time', 'datediff', 'relati...",5
4,Is there a standard way for a web server to be...,Determine a user's timezone,<html><browser><timezone><user-agent><timezone...,24,147,"['standard', 'way', 'web', 'server', 'abl', 'd...","['determin', ""user'"", 'timezon']","['html', 'browser', 'timezone', 'user-agent', ...",5
5,What is the difference between Math.Floor() an...,Difference between Math.Floor() and Math.Trunc...,<.net><math>,10,58,"['differ', 'math.floor()', 'math.truncate()', ...","['differ', 'math.floor()', 'math.truncate()']","['.net', 'math']",2
6,How do you expose a LINQ query as an ASMX web ...,Filling a DataSet or DataTable from a LINQ que...,<c#><linq><web-services><.net-3.5>,7,14,"['expos', 'linq', 'queri', 'asmx', 'web', 'ser...","['fill', 'dataset', 'datat', 'linq', 'queri', ...","['c#', 'linq', 'web-services', '.net-3.5']",4
7,How do I store binary data in MySQL?\n,Binary Data in MySQL,<mysql><database><binary-data><data-storage>,9,20,"['store', 'binari', 'data', 'mysql?']","['binari', 'data', 'mysql']","['mysql', 'database', 'binary-data', 'data-sto...",4
8,I'm looking for the fastest way to obtain the ...,What is the fastest way to get the value of π?,<performance><algorithm><language-agnostic><un...,23,80,"[""i'm"", 'look', 'fastest', 'way', 'obtain', 'v...","['fastest', 'way', 'get', 'valu', 'π?']","['performance', 'algorithm', 'language-agnosti...",5
9,If I have a trigger before the update on a tab...,Throw an error in a MySQL trigger,<mysql><database><triggers>,6,25,"['trigger', 'updat', 'table,', 'throw', 'error...","['throw', 'error', 'mysql', 'trigger']","['mysql', 'database', 'triggers']",3


## Unsupervised approach

### LDA with bag of words:

In [ ]:
no_topics = 20

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf_idf_model)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print (topic_idx)
        print ([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

no_top_words = 10
display_topics(lda, tf_idf_model, most_freq)


## Supervised approach

### Bag of Words

In [ ]:
list(nltk.bigrams(df.title_tokens[3]))

In [ ]:
df.t = df.tokens + df.title_tokens

In [ ]:
wordfreq = {}
for sentence in df.t:
    for token in sentence:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [ ]:
most_freq = heapq.nlargest(200, wordfreq, key=wordfreq.get)

In [ ]:
sentence_vectors = []
for sentence in df.t:
    sent_vec = []
    for token in most_freq:
        if token in sentence:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [ ]:
sentence_vectors = np.asarray(sentence_vectors)

In [ ]:
sentence_vectors

### TF-IDF

In [ ]:
word_idf_values = {}
for token in most_freq:
    doc_containing_word = 0
    for sentence in df.t:
        if token in sentence:
            doc_containing_word += 1
    word_idf_values[token] = np.log(len(df.t)/(1 + doc_containing_word))

In [ ]:
word_tf_values = {}
for token in most_freq:
    sent_tf_vector = []
    for sentence in df.t:
        doc_freq = 0
        for word in sentence:
            if token == word:
                  doc_freq += 1
        word_tf = doc_freq/len(sentence)
        sent_tf_vector.append(word_tf)
    word_tf_values[token] = sent_tf_vector

In [ ]:
tfidf_values = []
for token in word_tf_values.keys():
    tfidf_sentences = []
    for tf_sentence in word_tf_values[token]:
        tf_idf_score = tf_sentence * word_idf_values[token]
        tfidf_sentences.append(tf_idf_score)
    tfidf_values.append(tfidf_sentences)

In [ ]:
tf_model = np.asarray(word_tf_values)

In [ ]:
tf_idf_model = np.asarray(tfidf_values)

In [ ]:
tf_idf_model.shape